## Functions to compute multipole expansion of $r^LY^L_{\mu}(\Omega)$ in $SU(3)$ 

This functions are based in the expansion
$$
\left[r^{L_o}_{\sigma}Y^{L_o}_\mu(\hat{r}_{\sigma})\right]^{\hspace{3mm} L_o=J_o}_{\sigma\hspace{4mm}M_{J_o}} =\sqrt{\frac{2}{2L_o+1}}\sum_{(\lambda_o,\mu_o),K_o} \sum_{\eta,\eta'}\sum_{l,l'}(-1)^{\eta}\sqrt{2l'+1}\Big\langle \eta',l',\frac{1}{2} \Big|\Big| \hspace{2mm}\left[r^{L_o}_{\sigma}Y^{L_o}(\hat{r}_{\sigma})\right]^{\hspace{0mm} L_o=J_o} \Big|\Big| \eta,l,\frac{1}{2}\Big\rangle\times\langle (\eta',0),0,l'; (0,\eta),0,l||(\lambda_o,\mu_o),K_o,L_o \rangle_{\rho_o=1}\Big\{a^{\dagger}_{(\eta',0)\frac{1}{2}}\times \tilde{a}_{(0,\eta)\frac{1}{2}}\Big\}^{\rho_o=1(\lambda_o,\mu_o)K_oL_o=J_o}_{\hspace{20mm}M_{J_o}}
$$

$$
=\sum_{(\lambda_o,\mu_o),K_o}\sum_{\eta,\eta'}\mathcal{V}_{\sigma}(\eta',\eta,\lambda_o,\mu_o, K_o)\Big\{a^{\dagger}_{(\eta',0)\frac{1}{2}}\times \tilde{a}_{(0,\eta)\frac{1}{2}}\Big\}^{\rho_o=1(\lambda_o,\mu_o)K_oL_o=J_o}_{\hspace{20mm}M_{J_o}}
$$

In [ ]:
# Libraries
import numpy as np
import pandas as pd
import math
from sympy.physics.quantum.cg import CG

In [2]:
# Coefficients of radial expansion
def a(n,l,k):
    return (-1)**k/math.factorial(abs(k)) * np.sqrt( (2*math.factorial(n)) / (math.gamma(n+l+3/2)) )* (math.gamma(n+l+3/2)) / (math.factorial(n-k)*math.gamma(k+l+3/2))

# Coefficients B (n',l', n, l, p)
def B(n1,l1,n2,l2,p):
    s = 0
    for k in range(0, n2+1):
        if ( (p-k-1/2*(l1+l2) >= 0) and (n1-p+k+(1/2)*(l1+l2) >= 0) and (n2-k >= 0) ):
            s += a(n2,l2,k)*a(n1,l1,p-k-(1/2)*(l1+l2))

    return s*1/2*math.gamma(p+3/2)

# Matrix elements < n',l', m'| r^L | n, l, m >
def rL(L, n1, l1, n2, l2):
    s = 0
    for p in np.arange((l1+l2)/2, 1/2*(l1+l2)+n1+n2+1):
        s += B(n1,l1,n2,l2,p)*(math.gamma(p+3/2+L/2))/(math.gamma(p+3/2))
    return s
    
# 3-j symbol
def threeJsymbol(l1,m1,l2,m2,l3,m3):
    return float(CG(l1,m1,l2,m2,l3,-m3).doit())*((-1)**(l1-l2-m3))/np.sqrt(2*l3+1)
   
# Clebsch-Gordan coefficient
def C_G(l1,m1,l2,m2,l3,m3):
    return float(CG(l1,m1,l2,m2,l3,m3).doit())
    
# Matrix elements < n',l',m' | r^L Y_LM | n, l, m >
def rLYL(L, M, n1, l1, m1, n2, l2, m2):
    return (-1)**(m1)*np.sqrt( ((2*l2+1)*(2*l1+1)*(2*L+1))/(4*np.pi) )*threeJsymbol(l1,-m1,L,M,l2,m2)*threeJsymbol(l1,0,L,0,l2,0)*rL(L, n1, l1, n2, l2)

# Function to compute rme up to shell 7
def RME(L, N):           # Arguments: L: Angular momentum of tensor r^LY^L
                         #            N: List with values [n,l] to compute
    
    # List to store the non-zero matrix elements  
    nonZeroRME = []
    
    # Cycles over radial quantum numbers
    for i in range(0,len(N)):
        for j in range(0,len(N)):
            
            # Centinel value to store rmes
            centinel = 0
            
            # Cycles over projection quantum numbers
            for m1 in range(-1*N[i][0], N[i][0]+1):
                for m2 in range(-1*N[j][0], N[j][0]+1):
                    for M in range(-1*L,L+1):
                        
                        cg = C_G(N[j][0],m2,L,M,N[i][0],m1)
                        
                        if (cg != 0):
                            
                            me = rLYL(L, M, N[i][1],  N[i][0], m1 ,N[j][1],  N[j][0], m2)
                            
                            #print("<",2*N[i][1]+N[i][0],",", N[i][0],"||r",L,"Y",L,"||",2*N[j][1]+N[j][0],",", N[j][0],"> = ", me/cg, end='')
                            #print("\t<",N[j][0],",",m2,"; ",L,", ", M,"|",N[i][0],",",m1,"> = ", cg)  
                            
                            if (centinel == 0 and me != 0):
                                centinel = 1
                                nonZeroRME.append([[2*N[i][1]+N[i][0], N[i][0], 2*N[j][1]+N[j][0], N[j][0]], me/cg])
                            
                            #print(2*N[i][1]+N[i][0],",", N[i][0],"|| || ",2*N[j][1]+N[j][0], ",", N[j][0], end="")
                            #print("----", rLYL(L, M, N[i][1],  N[i][0], m1 ,N[j][1],  N[j][0], m2))
                            #print("----", rL(L, N[i][1],  N[i][0], N[j][1],  N[j][0]))
                            #print("----", threeJsymbol(N[j][0],-m1,L,M,N[i][0],m2))
                            #print("----", threeJsymbol(N[j][0],0,L,0,N[i][0],0))

                            
                        #else:
                        #    print("Cannot compute reduced matrix element: <",2*N[i][1]+N[i][0],",", N[i][0],"||r",L,"Y",L,"||",2*N[j][1]+N[j][0],",", N[j][0],">\t", end='')  
                        #    print("<",N[j][0],",",m2,"; ",L,", ", M,"|",N[i][0],",",m1,"> = ", C_G(N[j][0],m2,L,M,N[i][0],m1))                  
    return nonZeroRME

# Tests . B (n', l', n, l, p) with  MATRIX ELEMENTS IN NUCLEAR SHELL THEORY, Brody, Jacob, Moshinsky . 1960
#print(B(3, 4, 3, 4, 5))
#print(B(1, 7, 0, 9, 9))
#print(B(3, 1, 2, 3, 7))
#print(B(3, 6, 2, 8, 9))

In [3]:
# Values of [l,n] in shell N. Recall that N = 2n+l
N0 = [[0, 0]]
N1 = [[1, 0]]
N2 = [[2, 0], [0, 1]]
N3 = [[3, 0], [1, 1]]
N4 = [[4, 0], [2, 1], [0, 2]]
N5 = [[5, 0], [3, 1], [1, 2]]
N6 = [[6, 0], [4, 1], [2, 2], [0, 3]]
N7 = [[7, 0], [5, 1], [3, 2], [1, 3]]

## Import functions to implement different $SU(3)$ libraries

In [4]:
%run SU3_implementations.ipynb

In [5]:
# Tests 
#UNtoU3(5, [2,2,2,2])
#SU3prod(39, 2 , 31, 2)
#SU3WignerCoeff(5,0,-1,0,6,-1,5,6,-1)
#SU3WignerCoeff(60,8,2,1,1,2,60,8,-1)

In [6]:
def SU3_rLYL_expansion(shells, L):
    
    # Constant factor 
    factor = np.sqrt(2/(2*L+1))
    
    # Shell where the expansion will be computed
    N = []
    for i in shells:
        N += globals()[f"N{i}"]
    
    # Double bar reduced matrix elements
    rmes = RME(L, N)
    
    # Array for terms of the expansion
    terms = []

    # Computation of the terms in the sum
    for etap in shells:
        for eta in shells:
            
            # Coupling irreps
            l0m0 = SU3prod(etap, 0, 0, eta)
            
            for lp in range(etap,-1,-2):
                for l in range(eta,-1,-2):
                    for index, row in l0m0.iterrows():
                        
                        # Coupling coefficients
                        coeffs = SU3WignerCoeff(etap,0,lp,0,eta,l,row['lam'],row['mu'],L)
                        
                        if len(coeffs) > 0:
                            
                            for indexcoeff, rowcoeff in coeffs.iterrows():
                                
                                # SU(3) Wigner coefficient
                                c = rowcoeff["rho1"]
                                
                                # Loop over reduced matrix elements
                                for rme in rmes:
                                    if (rme[0][0] == etap and rme[0][1] == lp and rme[0][2] == eta and rme[0][3] == l):
                                        
                                        # Get rme
                                        m = rme[1]
                                        
                                        # Append the 
                                        terms.append([etap, eta, row['lam'], row['mu'], rowcoeff["k3"], factor*(-1)**eta*np.sqrt(2*lp+1)*m*c ])                             
                                            
                                        break
    
    # Dataframe for terms of the expansion
    expansion_df = pd.DataFrame(np.array(terms), columns = ["etap","eta","lam_o", "mu_o", "K_o", "V"])  
                             
    return pd.DataFrame(expansion_df.groupby(["etap","eta","lam_o", "mu_o", "K_o"])['V'].sum())